<h1, align=center>数据科学引论 - Python之道 </h1>

<h1, align=center>课程项目立项</h1>

<hr>
<div><h1 align=center><font style="font-style:italic;">企业年报信息与其经营情况以及未来股价的相关性分析</font></h1></div>
<div><h3 align=center>作者：<font style="font-style:italic;">陆树成</font></h3></div>

---
## 问题描述
#### 在本项目中，希望针对什么问题，提出什么样的解决方案。
包括：  
  1. 问题背景；  
  2. 问题阐述；  
  3. 项目预期达到的效果；  
  
  请将下面的代码框类型修改为Markdown，然后撰写以上内容

1. 问题背景

> 上市公司的年报中包含了大量的有价值数据，在分析该上市公司的经营情况的过程中具有举足轻重的作用。
>
> 上市公司往往会在年报中对公司当年与未来的经营战略进行阐释，而公司的经营战略会与公司目前的经营状况、行业的竞争压力、未来的发展前景等具备高度的相关关系。

2. 问题阐述

> 上市公司在年报中披露的公司经营战略是否与企业目前的经营状况或未来的盈利能力等因素存在相关关系，是否会与长远期的公司股价存在相关关系？

3. 项目预期达到的效果

> 得出年报披露的战略情况与某些经营状况、未来股价（变动）的关联关系。
>
> 例如：“行业竞争”与目前或未来的利润空间收窄存在相关关系、“低成本战略”与公司未来股价的涨跌幅之间的相关关系等

---
## 问题解决方案
#### 请你对你的解决方案进行描述。
包括：  
  1. 解决问题所需的数据集；  
  2. 数据集的收集方法；
  3. 问题解决方案的最终呈现形式。
  
  请将下面的代码框类型修改为Markdown，然后撰写以上内容

1. 解决问题所需的数据集

> 上市公司历年的年度报告PDF
>
> 上市公司历年的主要财务指标数据
>
> 上市公司历年的行情数据

2. 数据集的收集方法

> 在交易所官网（上交所、深交所）或相关的数据服务网（如巨潮资讯等）上爬取各上市公司历年的年度报告PDF文件与财务指标数据

3. 问题解决方案的最终呈现形式

> 最终将呈现年报中特定的公司战略文字信息与公司的股票收益、公司业绩情况、特定财务指标之间的统计关系/回归关系/机器学习训练结果，并得出相应的结论








---
## 数据收集练习
#### 运用本节课所学习到的内容，在网络上利用网络爬虫来收集数据。
要求：  
  1. 请你在下面的代码框中，运用本节课的知识，编写利用爬虫来获取你的项目需要用到的数据集的程序；  
  2. 如果你使用的是现成的数据集，那么请你编写利用爬虫来获取下面数据集的程序：   
     爬取下面这三个链接的网页中，列出的每一个Site的【名称、描述、网页地址】
     http://dmoztools.net/Computers/Programming/Languages/Python/Development_Tools/
     http://dmoztools.net/Computers/Programming/Languages/Python/Books/
     http://dmoztools.net/Computers/Programming/Languages/Python/Implementations/
  3. 无论你编写的程序爬取的是什么数据集，都应该把它们存储为.csv格式的文件，在后续课程的作业中可以复用这个数据集
  
  请在下面的代码框中编写你的程序。

In [2]:
import scrapy
import time
import json
import os
from scrapy.selector import Selector
import csv

class MySpider(scrapy.Spider):

    name = "spider"

    def __init__(self):

#         self.file = open('demo1_quotes.json', 'w')
        # 写入csv文件
        self.file = open('pythonLibrary.csv', 'w',
                         encoding='GBK', newline='')
        self.csvWriter = csv.DictWriter(self.file, fieldnames=["name", "description", "address"])
        self.csvWriter.writeheader()

        # 设置待爬取网站列表
        self.urls = []
        for i in ['Development_Tools','Books','Implementations']:
            self.urls.append('http://dmoztools.net/Computers/Programming/Languages/Python/%s/'%(i))

        print(self.urls)

    def start_requests(self):
        # self.init_urls()
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)

    # parse方法会在每个request收到response之后调用
    def parse(self, response):

        # 提取名言列表
        quotes = response.xpath('//div[@class="site-item "]').extract()
        for quote in quotes:
            # 重新将字符串还原为选择器
            quote=Selector(text=quote)
            # 网址
            address = quote.xpath ('//div[@class="title-and-desc"]/a/@href').extract_first()
            # 名称
            name = quote.xpath ('//div[@class="title-and-desc"]/a/div/text()').extract_first()
            name = name.replace('\t', '').replace('\n', '').replace('\r', '')
            # 描述
            description = quote.xpath ('//div[@class="title-and-desc"]/div/text()').extract_first()
            description = description.replace('\t', '').replace('\n', '').replace('\r', '').strip()
            # 构建字典对象
            item = {"name": name, "description": description, "address": address}
            # 将每个条目写入文件
#             self.file.write(line + "\n")
            self.csvWriter.writerow(item)
            
        # 及时将内容写入文件，否则可能会出现少许延迟
        self.file.flush()
        os.fsync(self.file)
        # 输出当前解析完成的网页网址，可以当做爬取进度来看待,与程序逻辑无关
        print("over: " + response.url)

In [3]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MySpider)
process.start()  # 这句代码就是开始了整个爬虫过程 ，会输出一大堆信息，可以无视

2019-11-08 16:15:32 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-11-08 16:15:32 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.6.1, Platform Windows-10-10.0.18362-SP0
2019-11-08 16:15:32 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-11-08 16:15:32 [scrapy.extensions.telnet] INFO: Telnet Password: 44cae8a270334d6f
2019-11-08 16:15:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']


['http://dmoztools.net/Computers/Programming/Languages/Python/Development_Tools/', 'http://dmoztools.net/Computers/Programming/Languages/Python/Books/', 'http://dmoztools.net/Computers/Programming/Languages/Python/Implementations/']


2019-11-08 16:15:33 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2019-11-08 16:15:33 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.ref

over: http://dmoztools.net/Computers/Programming/Languages/Python/Implementations/
over: http://dmoztools.net/Computers/Programming/Languages/Python/Development_Tools/
over: http://dmoztools.net/Computers/Programming/Languages/Python/Books/


In [5]:
import pandas as pd
pd.read_csv('pythonLibrary.csv')

name  \
0                                            CPython    
1                                         IronPython    
2                                             Jython    
3                                    Python for .NET    
4                                   Stackless Python    
5                         Hap Python Remote Debugger    
6                PyCrust - The Flakiest Python Shell    
7                                           Pymerase    
8                                          PyReverse    
9                     Tixapps Applications Framework    
10                                            Winpdb    
11  Data Structures and Algorithms with Object-Ori...   
12                                Dive Into Python 3    
13         Foundations of Python Network Programming    
14                                 Free Python books    
15          FreeTechBooks: Python Scripting Language    
16  How to Think Like a Computer Scientist: Learni...   
17                         An Introduction to Python    
18                              Making Use of Python    
19                                  Practical Python    
20                  Pro Python System Administration    
21          Programming in Python 3 (Second Edition)    
22                                  Python 2.1 Bible    
23              Python 3 Object Oriented Programming    
24                  Python Language Reference Manual    
25  Python Programming with the Java Class Librari...   
26            Sams Teach Yourself Python in 24 Hours    
27                         Text Processing in Python    
28                        XML Processing with Python    

                                          description  \
0   The original implementation of Python, written...   
1     A fast Python implementation for .NET and Mono.   
2   A 100 % Pure Java implementation of the Python...   
3   Package gives nearly seamless integration with...   
4   An experimental implementation that supports c...   
5   A remote debugger and IDE that can also be use...   
6   An interactive, graphical Python shell written...   
7   A tool intended to generate a python object mo...   
8   A set of tools for reverse engineering Python ...   
9   Tixapps applications are a suite of developmen...   
10  An advanced python debugger, with support for ...   
11  The primary goal of this book is to promote ob...   
12  By Mark Pilgrim, Guide to Python 3  and its di...   
13  This book covers a wide range of topics. From ...   
14                   Free Python books and tutorials.   
15  Annotated list of free online books on Python ...   
16  By Allen B. Downey, Jeffrey Elkner, Chris Meye...   
17  By Guido van Rossum, Fred L. Drake, Jr.; Netwo...   
18  By Rashi Gupta; John Wiley and Sons, 2002, ISB...   
19  By Magnus Lie Hetland; Apress LP, 2002, ISBN 1...   
20  By Rytis Sileika, ISBN13: 978-1-4302-2605-5, U...   
21           A Complete Introduction to the Python 3.   
22  By Dave Brueck, Stephen Tanner; John Wiley and...   
23  A step-by-step tutorial for OOP in Python 3, i...   
24  By Guido van Rossum, Fred L. Drake, Jr.; Netwo...   
25  By Richard Hightower; Addison-Wesley, 2002, 02...   
26  By Ivan Van Laningham; Sams Publishing, 2000, ...   
27  By David Mertz; Addison Wesley. Book in progre...   
28  By Sean McGrath; Prentice Hall PTR, 2000, ISBN...   

                                              address  
0                    https://www.python.org/download/  
1   http://www.codeplex.com/wikipage?ProjectName=I...  
2             http://sourceforge.net/projects/jython/  
3                   http://pythonnet.sourceforge.net/  
4                           http://www.stackless.com/  
5                 http://hapdebugger.sourceforge.net/  
6            http://sourceforge.net/projects/pycrust/  
7                    http://pymerase.sourceforge.net/  
8                        http://www.logilab.org/2560/  
9                 http://tix.sourceforge.net/Tixapps/  
10         